In [1]:
import numpy as np
import scipy.misc
import tensorflow as tf
import pickle as p # python2系はcPickle
from keras.utils import np_utils
from keras.models import Sequential, Model, model_from_json
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.datasets import cifar10
# import keras.backend.tensorflow_backend as KTF
import matplotlib.pyplot as plt

# Cifar10データセット
from keras.datasets import cifar10

(x_train,y_train),(x_test,y_test)=cifar10.load_data()
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

170500096/170498071 [==============================] - 14s 0us/step
(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [2]:
def change3class(exp_val,obj_val):
  new_exp=[]
  new_obj=[]
  for (x,y) in zip(exp_val,obj_val):
    if y==3:
      new_exp.append(x)
      new_obj.append(0)
    elif y==5:
      new_exp.append(x)
      new_obj.append(1)
    elif y==1:
      new_exp.append(x)
      new_obj.append(2)
    else:
      pass
  new_exp=np.array(new_exp)
  return new_exp,new_obj

In [3]:
in_shape=(32,32,3)
cls_num=3

def build_model():
  model=Sequential()

  model.add(Conv2D(32,(3,3),padding='same',input_shape=in_shape))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())
  model.add(Dropout(0.2))

  model.add(Conv2D(64,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())
  model.add(Dropout(0.3))

  model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
  model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
  model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
  model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
  model.add(MaxPooling2D())
  model.add(Dropout(0.4))

  model.add(Flatten())
  model.add(Dense(100))
  model.add(Activation('relu'))

  model.add(Dropout(0.5))
  model.add(Dense(cls_num))
  model.add(Activation('softmax'))

  model.compile(loss="categorical_crossentropy",
        metrics   = ["accuracy"],
        optimizer = "Adam"
  )

  return model

In [4]:
if __name__ == "__main__":
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)

    X_train, y_train = change3class(X_train, y_train)
    X_test, y_test = change3class(X_test, y_test)

    y_train = np_utils.to_categorical(y_train)
    y_test  = np_utils.to_categorical(y_test)

    X_train = X_train.astype('float32')
    X_test  = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0

    print("X_train", X_train.shape)
    print("y_train", y_train.shape)
    print("X_test", X_test.shape)
    print("y_test", y_test.shape)

    model = build_model()
    model.fit(X_train, y_train,
        epochs=30,
        batch_size=128,
        validation_data=(X_test, y_test)
    )

    json_string = model.to_json()
    open('test.json', 'w').write(json_string)
    model.save_weights('test.hdf5')

    # evaluate
    score = model.evaluate(X_test, y_test)
    print("test loss", score[0])
    print("test acc",  score[1])

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)
X_train (15000, 32, 32, 3)
y_train (15000, 3)
X_test (3000, 32, 32, 3)
y_test (3000, 3)
Epoch 1/30
118/118 [==============================] - 4s 32ms/step - loss: 0.8171 - accuracy: 0.5571 - val_loss: 0.6697 - val_accuracy: 0.6390
Epoch 2/30
118/118 [==============================] - 3s 28ms/step - loss: 0.6043 - accuracy: 0.6824 - val_loss: 0.5397 - val_accuracy: 0.7137
Epoch 3/30
118/118 [==============================] - 3s 28ms/step - loss: 0.5573 - accuracy: 0.7111 - val_loss: 0.5427 - val_accuracy: 0.7303
Epoch 4/30
118/118 [==============================] - 3s 28ms/step - loss: 0.5349 - accuracy: 0.7362 - val_loss: 0.4852 - val_accuracy: 0.7607
Epoch 5/30
118/118 [==============================] - 3s 28ms/step - loss: 0.4949 - accuracy: 0.7548 - val_loss: 0.4562 - val_accuracy: 0.7720
Epoch 6/30
118/118 [==============================] - 3s 28ms/step - loss: 0.4775 - accuracy: 0.7730 - val_loss: 0.4676 - val_accuracy: 0

In [5]:
import keras
import sys,os
import scipy
import numpy as np
from keras.models import model_from_json
from PIL import Image

import json

In [7]:
imsize=(32,32)

testpic='./testpic/'
keras_model='./test.json'
keras_param='./test.hdf5'

def load_image(path):
  img=Image.open(path)
  img=img.convert('RGB')
  img=img.resize(imsize)

  img=np.asarray(img).astype(float)
  img/=255.0
  return img

def get_file(dir_path):
  filenames=os.listdir(dir_path)
  return filenames

if __name__=='__main__':
  pic=get_file(testpic)

  model=model_from_json(open(keras_model).read())
  model.load_weights(keras_param)
  model.summary()

  for pic_name in pic:
    print(pic_name)
    img=load_image(testpic+pic_name)
    prd=model.predict(np.array([img]))
    print(prd)
    prelabel=np.argmax(prd)

    if prelabel == 0:
            print(">>> 猫")
    elif prelabel == 1:
            print(">>> 犬")
    elif prelabel == 2:
            print(">>> 乗り物")

    print("-"*55)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        3